# Differential cross section fitting using BRICK 
## $^{12}\rm{C}(^{12}C, \alpha_{0})^{20}\rm{Ne}$ 

Minimizing $\chi^2$ version

## Steps
* Construct a Python object that enables the calling of AZURE2 with an arbitrary input vector $\theta$.
* Read the data files.
* Construct a global function that reads a SRIM file, input energy, and return the -dE/dx.
* Construct the yield function.
* Construct a $\chi^2$ function.
* Minimization.
* Preview the results.

In [ ]:
import os
import emcee
import corner

import numpy as np
from scipy.interpolate import interp1d
from scipy.integrate import quad, quadrature
# import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import clear_output
from lmfit import Parameters, Minimizer
from iminuit import Minuit

import re
from tqdm import tqdm
from scipy import stats
from brick.azr import AZR
from multiprocess import Pool

## Step 1
* Construct a Python object that enables the calling of AZURE2 with an arbitrary input vector $\theta$.

In [ ]:
# Restrict processes to one thread only. Good for AZURE2
os.environ['OMP_NUM_THREADS'] = '1'

# Construct an object that calls AZURE2 and takes input 'a0.arz'
azr = AZR('a0.arz')
azr.root_directory = 'tmp/'

# How many parameters are going to be sampled?
nd = azr.config.nd + 2
print(f'There are {nd} sampled parameters.\n')

# Get the initial input parameters
# theta0 = azr.config.get_input_values()

# Read initial parameters from previously fitted results.
theta0 = []
theta0_mask = []
# initial_params_file = "results/best-emcee.txt"
# with open(initial_params_file, "r") as file:
#     for line in file:
#         # Split the line by ":"
#         parts = line.strip().split(":")
#         # Extract the numeric expression (second part after ":")
#         expression = parts[1].strip()
#         # Split the expression into terms
#         # Example: "9.07146 + 3.37034 - 3.77688" becomes ["9.07146", "3.37034", "3.77688"]
#         expression = expression.replace("+", " ").replace("-", " ").split()
#         # Convert to floats
#         base_value = float(expression[0])  # First number (base value)
#         addition_value = float(expression[1])  # Second number (value after "+")
#         subtraction_value = float(expression[2])  # Third number (value after "-")
#         # Append the base value to column_2_list
#         theta0.append(base_value)
#         # Find the max between addition_value and subtraction_value
#         theta0_mask.append(100.*max(addition_value, subtraction_value))

for i in range(nd-2):
    theta0.append(100)
theta0.append(6E4)
theta0.append(1e-17)

print(f'The initial input parameters are:\n {theta0}\n')
print(f'The initial input parameters width are:\n {theta0_mask}\n')

# What labels have been assigned to those parameters?
labels = azr.config.labels
labels.append('$N_{v}$')
labels.append('const')
print(f'The labels of the input parameters are:\n {labels}\n')

## Step 2
* Read the data files.

In [ ]:
# Read in the data files
data_files_list = os.listdir('data/')
data_files_list.sort()
print(data_files_list)
num_energies = len(data_files_list)
energies = [float(filename.replace('.txt', '')) for filename in data_files_list]
energies = np.array(energies)
data = []
angle_lab = []
yield_lab = []
yield_err = []
for i in range(num_energies):
    temp = np.loadtxt("data/"+data_files_list[i])
    temp[:,2]=temp[:,2]*1E-15 # Scale factor of the yield
    temp[:,3]=temp[:,3]*1E-15 # Scale factor of the yield's error
    data.append(temp)
    angle_lab.append(temp[:,1])
    yield_lab.append(temp[:,2])
    yield_err.append(temp[:,3])
num_angles = len(angle_lab[0])
angle_lab = np.asarray(angle_lab, dtype='float32').T
yield_lab = np.asarray(yield_lab, dtype='float32').T
yield_err = np.asarray(yield_err, dtype='float32').T
print('shape of yield_lab is', yield_lab.shape)

# Take a look at the data
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
for i in range(num_energies):
    ax.errorbar(angle_lab[:,i], yield_lab[:,i], yerr=yield_err[:,i], linestyle='', capsize=2, fmt="o", label=data_files_list[i])
    # ax.scatter(angle_lab[i], yield_lab[i])
ax.legend()
ax.set_xlim(0, 180)
ax.set_xlabel(r'$\theta$ (deg, lab)')
ax.set_ylabel(r'Yields (arb. units)')

## Step 3
* Construct a global function that reads a SRIM file, input energy, and return the -dE/dx.

In [ ]:
# Read in the SRIM table
srim_file = 'srim/CarbonInCarbon.txt'
srim_table = []
with open(srim_file, 'r') as file:
    for line in file:
        cols = line.split()
        srim_table.append((float(cols[0]), float(cols[1]))) # (MeV, MeV/mm)
srim_table=np.array(srim_table)

# Construct the interpolation
srim_interp = interp1d(srim_table[:,0], srim_table[:,1],kind='cubic')
# new_x = np.arange(0.1,50,0.1)
# plt.figure()
# plt.plot(srim_table[:,0], srim_table[:,1], 'o', label='SRIM table')
# plt.plot(new_x, srim_interp(new_x), '-', label='Interpolated -dE/dX')
# plt.xlabel('E (MeV)')
# plt.ylabel('-dE/dx (MeV/mm)')
# plt.title('Stopping power of carbon in carbon')
# plt.legend()

## Step 4
* Construct the yield function.

In [ ]:
# Convert COM to Lab
m_a = 12
m_A = 12
m_b = 4
m_B = 20
Q_value=4.62
# gamma = np.sqrt((m_a*m_b)/(m_A*m_B)*(m_b+m_B)/(m_a+m_A)*E_COM[0]/(E_COM[0]+Q))
def yield_angle(theta, angle_index):
    output = azr.extrapolate(theta[:-2],[angle_index])[0]
    energy_c = output[:,0]
    angle_c = output[:,2]
    sigma_c = output[:,3]
    energy_l=(m_a+m_A)/m_A*energy_c
    gamma_local = np.sqrt((m_a*m_b)/(m_A*m_B)*(m_b+m_B)/(m_a+m_A)*energy_c/(energy_c+Q_value))
    sigma_l = (1+gamma_local**2.+2.*gamma_local*np.cos(angle_c/180.*np.pi))**(3./2.)/(1+gamma_local*np.cos(angle_c/180.*np.pi))*sigma_c
    # a_l=np.arccos((gamma_local+np.cos(angle_c/180*np.pi))/np.sqrt(1+gamma_local**2+2*gamma_local*np.cos(angle_c/180*np.pi)))
    sigma_interp = interp1d(energy_l, sigma_l, kind='linear')
    def integrand(E):
        return sigma_interp(E)/srim_interp(E)
    res = np.zeros(num_energies)
    res[0] = quad(integrand, 0.5, energies[0]*m_A/(m_a+m_A))[0] # Integrate from E = 0.5 to 4.4 MeV
    for i in range(num_energies):
        if(i==0):
            continue
        else:
            res[i] = res[i-1]+quad(integrand, energies[i-1]*m_A/(m_a+m_A), energies[i]*m_A/(m_a+m_A))[0] # Integrate from E_{i-1} to E_{i}
    return res * theta[-2] + theta[-1]

## Step 5
* Construct a $\chi^2$ function.
$$\chi^{2}=\sum_{i}\frac{(y_{i}-\mu_{i})^{2}}{\sigma_{i}^{2}}$$

In [ ]:
def get_chi2(theta):
    args = range(num_angles)
    with Pool(processes=24) as pool:
        mu = pool.starmap(yield_angle, [(theta, i) for i in args])
    mu = np.array(mu, dtype='float32')
    temp_sum = np.zeros(num_energies)
    for i in range(num_energies):
        if(energies[i] == 4.4):
            temp_sum[i]=np.sum(np.power(((yield_lab[[8,13,14,16,19,21],i] - mu[[8,13,14,16,19,21],i])/yield_err[[8,13,14,16,19,21],i]), 2.))
        elif(energies[i] == 4.6):
            temp_sum[i]=np.sum(np.power(((yield_lab[[1,2,12,14,15,16,19,20,21,22],i] - mu[[1,2,12,14,15,16,19,20,21,22],i])/yield_err[[1,2,12,14,15,16,19,20,21,22],i]), 2.))
        elif(energies[i] == 4.7):
            exclude_rows = [9,23]
            temp_yield_err = np.delete(yield_err, exclude_rows, axis=0)
            temp_yield_lab = np.delete(yield_lab, exclude_rows, axis=0)
            temp_mu = np.delete(mu, exclude_rows, axis=0)
            temp_sum[i]=np.sum(np.power(((temp_yield_lab[:,i] - temp_mu[:,i])/temp_yield_err[:,i]), 2.))
        elif(energies[i] == 5.0):
            exclude_rows = [23]
            temp_yield_err = np.delete(yield_err, exclude_rows, axis=0)
            temp_yield_lab = np.delete(yield_lab, exclude_rows, axis=0)
            temp_mu = np.delete(mu, exclude_rows, axis=0)
            temp_sum[i]=np.sum(np.power(((temp_yield_lab[:,i] - temp_mu[:,i])/temp_yield_err[:,i]), 2.))
        elif(energies[i] == 5.1):
            exclude_rows = [23]
            temp_yield_err = np.delete(yield_err, exclude_rows, axis=0)
            temp_yield_lab = np.delete(yield_lab, exclude_rows, axis=0)
            temp_mu = np.delete(mu, exclude_rows, axis=0)
            temp_sum[i]=np.sum(np.power(((temp_yield_lab[:,i] - temp_mu[:,i])/temp_yield_err[:,i]), 2.))
        elif(energies[i] == 5.5):
            exclude_rows = [22,23]
            temp_yield_err = np.delete(yield_err, exclude_rows, axis=0)
            temp_yield_lab = np.delete(yield_lab, exclude_rows, axis=0)
            temp_mu = np.delete(mu, exclude_rows, axis=0)
            temp_sum[i]=np.sum(np.power(((temp_yield_lab[:,i] - temp_mu[:,i])/temp_yield_err[:,i]), 2.))
        elif(energies[i] == 5.7):
            exclude_rows = [23]
            temp_yield_err = np.delete(yield_err, exclude_rows, axis=0)
            temp_yield_lab = np.delete(yield_lab, exclude_rows, axis=0)
            temp_mu = np.delete(mu, exclude_rows, axis=0)
            temp_sum[i]=np.sum(np.power(((temp_yield_lab[:,i] - temp_mu[:,i])/temp_yield_err[:,i]), 2.))
        elif(energies[i] == 5.8):
            exclude_rows = [23]
            temp_yield_err = np.delete(yield_err, exclude_rows, axis=0)
            temp_yield_lab = np.delete(yield_lab, exclude_rows, axis=0)
            temp_mu = np.delete(mu, exclude_rows, axis=0)
            temp_sum[i]=np.sum(np.power(((temp_yield_lab[:,i] - temp_mu[:,i])/temp_yield_err[:,i]), 2.))
        elif(energies[i] == 6.0):
            exclude_rows = [0]
            temp_yield_err = np.delete(yield_err, exclude_rows, axis=0)
            temp_yield_lab = np.delete(yield_lab, exclude_rows, axis=0)
            temp_mu = np.delete(mu, exclude_rows, axis=0)
            temp_sum[i]=np.sum(np.power(((temp_yield_lab[:,i] - temp_mu[:,i])/temp_yield_err[:,i]), 2.))
        elif(energies[i] == 6.4):
            exclude_rows = [23]
            temp_yield_err = np.delete(yield_err, exclude_rows, axis=0)
            temp_yield_lab = np.delete(yield_lab, exclude_rows, axis=0)
            temp_mu = np.delete(mu, exclude_rows, axis=0)
            temp_sum[i]=np.sum(np.power(((temp_yield_lab[:,i] - temp_mu[:,i])/temp_yield_err[:,i]), 2.))
        elif(energies[i] == 6.6):
            exclude_rows = [23]
            temp_yield_err = np.delete(yield_err, exclude_rows, axis=0)
            temp_yield_lab = np.delete(yield_lab, exclude_rows, axis=0)
            temp_mu = np.delete(mu, exclude_rows, axis=0)
            temp_sum[i]=np.sum(np.power(((temp_yield_lab[:,i] - temp_mu[:,i])/temp_yield_err[:,i]), 2.))
        elif(energies[i] == 7.4):
            exclude_rows = [0]
            temp_yield_err = np.delete(yield_err, exclude_rows, axis=0)
            temp_yield_lab = np.delete(yield_lab, exclude_rows, axis=0)
            temp_mu = np.delete(mu, exclude_rows, axis=0)
            temp_sum[i]=np.sum(np.power(((temp_yield_lab[:,i] - temp_mu[:,i])/temp_yield_err[:,i]), 2.))
        else:
            temp_sum[i]=np.sum(np.power(((yield_lab[:,i] - mu[:,i])/yield_err[:,i]), 2.))
    return np.sum(temp_sum)

## Step 6
* Minimization

In [ ]:
ndf = 24.*28.-nd
iteration_count = 0
reduced_chi2 = []
iterations = []
def func(theta):
    theta = np.array(theta)
    residuals = get_chi2(theta)
    global iteration_count
    iteration_count = iteration_count+1

    reduced_chi2.append(residuals/ndf)
    iterations.append(iteration_count)

    if( iteration_count % 100 == 0 ): 
        print("Iteration: {:6d} it Chi2: {:15.9f} Chi2/ndf: {:15.9f}".format( iteration_count, residuals, residuals/ndf ), end="\r")
        clear_output(wait=True)
        plt.figure(figsize=(16,6))
        plt.plot(iterations[-500:], reduced_chi2[-500:])
        plt.xlabel('Iteration')
        plt.ylabel(r'$\chi^2$/ndf')
        plt.title('Chi2/ndf vs Iteration (in progress)')
        plt.grid(True)
        plt.show()

    return residuals

initial_guess = np.array( theta0 )
params = Parameters()
for i in range(nd):
    params.add( "param_{}".format(i), value=initial_guess[i], vary=True, min=0.0, max=np.inf )

mini = Minuit( func, params )
mini.errordef = 1 # chi2 minimization
mini.limits = [(0, None) for i in range(nd)]

print("Initial:", mini.values)

mini.simplex()
mini.migrad()
# mini.hesse(ncall=10)

# Print results
print("Valid:", mini.valid)
print("Accurate:", mini.accurate)
print("Best fit values:", mini.values)
print("Parameter errors:", mini.errors)


## Step 7
* Save the results

In [ ]:
# access the minimization results
fitted_params = np.array(mini.values)
param_errors  = np.array(mini.errors)
cov_matrix    = np.array(mini.covariance)
chi2_final = mini.fval
function_calls = mini.nfcn

# Save parameters to file
with open( "results/best-leastsq.txt", "w" ) as f:
    # Write a header with the chi2 and chi2_red, number of iterations, number of data and number of parameters
    f.write("# Chi2 = {} \n".format(chi2_final))
    f.write("# Chi2_reduced = {} \n".format(chi2_final/ndf))
    f.write("# Iterations = {} \n".format(function_calls))
    f.write("# Number of data = {} \n".format(24*28))
    f.write("# Number of parameters = {} \n".format(nd))
    for i in range( nd ):
        f.write( f"{azr.config.labels[i]}: {fitted_params[i]:.5e} +- {param_errors[i]:.5e}\n" )

# Save the covariance matrix
with open( "results/covariance-leastsq.txt", "w" ) as f:
    for i in range(nd):
        for j in range(nd):
            f.write("{:15.6f} ".format(cov_matrix[i,j]))
        f.write("\n")

## Step 8
* Resampling

In [ ]:
def s_factor(theta):
    output = np.array(azr.extrapolate(theta[:-2],[25])[0])
    s_factor_output = np.array([output[:,0], output[:,1], output[:,3], output[:,4]])
    return s_factor_output

def yield_angles(theta):
    mu=[]
    for i in range(num_angles):
        mu.append(yield_angle(theta, i))
    mu = np.array(mu, dtype='float32')
    return mu

flat_samples = np.random.multivariate_normal( fitted_params, cov_matrix, 500 )
# flat_samples = np.random.multivariate_normal( fitted_params, np.diag([1e-100]*57), 500 )
def process_parallel():
    # Shuffle and take last 100 samples
    np.random.shuffle(flat_samples)
    samples_to_process = flat_samples[-500:]
    
    # Create a pool of workers
    with Pool(processes=24) as pool:
        # Map the function to the samples and show progress
        buckets_s_factor = list(tqdm(
            pool.imap(s_factor, samples_to_process),
            total=len(samples_to_process)
        ))
        buckets_yield_angles = list(tqdm(
            pool.imap(yield_angles, samples_to_process),
            total=len(samples_to_process)
        ))
    
    return buckets_s_factor, buckets_yield_angles
buckets_s_factor, buckets_yield_angles = process_parallel()

## Step 9
* Plot the reampled results and compared it to the data

In [ ]:
buckets_s_factor=np.array(buckets_s_factor)

mean = np.percentile(buckets_s_factor, 50, axis=0 ).T
low = np.percentile(buckets_s_factor, 16, axis=0).T
high = np.percentile(buckets_s_factor, 84, axis=0).T

fig, ax = plt.subplots(1,2,figsize=( 15, 5 ))
ax[0].set_xlim(13, 20)
ax[0].set_yscale('log')
ax[0].set_xlabel(r'Excitation energy (MeV, COM)')
ax[0].set_ylabel(r'Cross section (barn)')
ax[0].plot(mean[20:,1], mean[20:,2], label="Extrapolation", color="red")
ax[0].fill_between(mean[20:,1], low[20:,2], high[20:,2], color="red", alpha=0.5 )

# ax[0].set_xlim(0, 5)
# ax[0].set_yscale('log')
# ax[0].set_xlabel(r'Ecm (MeV, COM)')
# ax[0].set_ylabel(r'S-factor (MeV b)')
# ax[0].plot(mean[:,0], mean[:,2]*mean[:,0]*np.exp(87.21/np.sqrt(mean[:,0])+0.46*mean[:,0]), label="Extrapolation", color="red")
# ax[0].fill_between(mean[:,0], low[:,2]*mean[:,0]*np.exp(87.21/np.sqrt(mean[:,0])+0.46*mean[:,0]), high[:,2]*mean[:,0]*np.exp(87.21/np.sqrt(mean[:,0])+0.46*mean[:,0]), color="red", alpha=0.5 )

ax[1].set_xlim(0, 5)
ax[1].set_yscale('log')
ax[1].set_xlabel(r'Ecm (MeV, COM)')
ax[1].set_ylabel(r'S-factor (MeV b)')
ax[1].plot(mean[:,0], mean[:,3], label="Extrapolation", color="red")
ax[1].fill_between(mean[:,0], low[:,3], high[:,3], color="red", alpha=0.5 )


In [ ]:
buckets_yield_angles=np.array(buckets_yield_angles)

mean = np.percentile(buckets_yield_angles, 50, axis=0 )
low = np.percentile(buckets_yield_angles, 16, axis=0)
high = np.percentile(buckets_yield_angles, 84, axis=0)

fig, ax = plt.subplots( int(num_energies / 2)+1, 2, figsize=( 15, 50 ) )
for i in range( num_energies ):
        ax[i//2, i%2].set_xlabel( "Angle (deg, lab)" )
        ax[i//2, i%2].set_ylabel( "Yield (arb. unit)" )

        mask = yield_err[:, i] > 0
        ax[i//2, i%2].errorbar(angle_lab[mask, i], yield_lab[mask, i], yerr=yield_err[mask, i], linestyle='', capsize=2, fmt="o", label=data_files_list[i])
        ax[i//2, i%2].legend()

        ax[i//2, i%2].plot(angle_lab[mask, i], mean[mask, i], marker="o", label="Extrapolation", color="red")
        ax[i//2, i%2].fill_between(angle_lab[mask, i], low[mask, i], high[mask, i], color="red", alpha=0.5 )
